# Create Public School Machine Learning Datasets
** This program creates all the _ML datasets in the NCEA repository.** 
* This notebook reads each School Dataset file located at \EducationDataNC\ *schoolYear* \School Datasets\ as input data.
* Different school years are processed by changing the *schoolYear* parameter.
* Different input / output files are processed / created by changing the *inputFileName* paramter in the cell below.  
* While a single program is used to create all the _ML datasets, one program copy per dataset is maintained in the repositiory so the dataset specific tranformation reports may be reviewed. 

**Datasets ending in ML are preprocessed for Machine Learning and go through the following transformations: **
1. Missing student body racial compositions are imputed using district averages.
2. Columns that have the same value in every single row are deleted.
3. Columns that have a unique value in every single row (all values are different) are deleted.
4. Empty columns (all values are NA or NULL) are deleted.
5. Numeric columns with more than the percentage of missing values specified by the *missingThreshold* parameter.
6. Remaining numeric, non-race columns with missing values are imputed / populated with 0.  In many cases, schools are not reporting values when they are zero. However, mean imputation or some other more sophisticated strategy might be considered here.
7. Categorical / text based columns with > *uniqueThreshold* unique values are deleted.
8. All remaining categorical / text based columns are one-hot encoded.  In categorical columns, one-hot encoding creates one new boolean / binary field per unique value in the target column, converting all categorical columns to a numeric data type. 
9. Duplicated or highly similar columns with > 95% correlation are delelted.    

In [21]:
#import required Libraries
import pandas as pd
import numpy as np
import os
import string

#**********************************************************************************
# Set the following variables before running this code!!!
#**********************************************************************************
#All raw data files are processed for the year below
schoolYear = 2019

#Location where copies of the raw data files will be read in from csv files.
dataDir = 'C:/Users/Jake/Documents/GitHub/EducationDataNC/' + str(schoolYear) + '/School Datasets/'

#Name of the file to be processed
#inputFileName = 'PublicSchools2019'
#inputFileName = 'PublicHighSchools2019'
#inputFileName = 'PublicMiddleSchools2019'
inputFileName = 'PublicElementarySchools2019'

#Input file being transformed for machine learning 
inputFile = dataDir + inputFileName + '.csv'

#Location where the new school datasets will be created.
outputDir = 'C:/Users/Jake/Documents/GitHub/EducationDataNC/' + str(schoolYear) + '/Machine Learning Datasets/'

#Missing Data Threshold (Per Column)
missingThreshold = 0.60

#Unique Value Threshold (Per Column)
#Delete Columns >  uniqueThreshold unique values prior to one-hot encoding. 
#(each unique value becomes a new column during one-hot encoding)
uniqueThreshold = 200

#Read in the School Data File
schoolData = pd.read_csv(inputFile, low_memory=False, dtype={'agency_code': object})
print('*********Start: Beginning Column and Row Counts********************************************')
schoolData.info(verbose=False)

#Select only public schools as charter schools are missing data for many columns.
schoolData = schoolData[(schoolData['designation_type'] == 'P')] #& (schoolData['avg_student_num'] > 0)] -rcd_admin missing

print('\r\n*********After: Selecting Only Public School Campuses**********************************')
schoolData.info(verbose=False)

#Save primary key
agency_code = schoolData['agency_code']
#Convert zip code to string
schoolData['zip'] = schoolData['zip'].astype('object')

*********Start: Beginning Column and Row Counts********************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1636 entries, 0 to 1635
Columns: 1485 entries, agency_code to welcome_url
dtypes: float64(1436), int64(1), object(48)
memory usage: 18.5+ MB

*********After: Selecting Only Public School Campuses**********************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1472 entries, 2 to 1635
Columns: 1485 entries, agency_code to welcome_url
dtypes: float64(1436), int64(1), object(48)
memory usage: 16.7+ MB


# Prepare Consolidated Dataset for Machine Learning
**Below we perform operations on the entire dataset to remove columns and update row values that could cause problems during machine learning.**

## Student Body Racial Composition Features 
**Impute / update missing Student Body Racial Composition Fields using mean imputation.**
* When there are no racial composition percentages for a particular school campus / agency_code, fill in the missing values 

In [22]:
#Get Student Body Racial Composition Fields
raceCompositionFields = schoolData.filter(regex='Indian|Asian|Hispanic|Black|White|PacificIsland|TwoOrMore|Minority')\
                                  .filter(regex='Pct').columns
    
rowsBefore = schoolData[raceCompositionFields].isnull().T.any().T.sum()

#Update missing race values with the district average when avaiable (No district averages for charter schools) 
schoolData[raceCompositionFields] = schoolData.groupby('lea_code')[raceCompositionFields]\
                                              .transform(lambda x: x.fillna(x.mean()))

    #Review dataset contents after Racial Composition Imputation
print('*********After: Updating Missing Racial Compostion Values****************************')   
rowsAfter = schoolData[raceCompositionFields].isnull().T.any().T.sum()
rowsUpdated = rowsBefore - rowsAfter
print('Rows Updated / Imputed: ', rowsUpdated) 
print('\r\nTotal Rows Missing Racial Compositions By District Code') 
schoolData['lea_code'][schoolData[raceCompositionFields].isnull().T.any().T].value_counts()

*********After: Updating Missing Racial Compostion Values****************************
Rows Updated / Imputed:  4

Total Rows Missing Racial Compositions By District Code


010LEA    21
070LEA     8
020LEA     7
090LEA     6
040LEA     6
060LEA     5
080LEA     4
030LEA     3
050LEA     3
Name: lea_code, dtype: int64

## Remove Columns with Problematic Data
**Here we remove entire columns that could cause problems during machine learning.  The following operations are performed:**
* Remove any columns that have the same value in every single row.
* Remove any columns that have a unique value in every single row (all values are different).
* Remove empty columns (all values are NA or NULL).

In [23]:
#Remove any fields that have the same value in all rows
UniqueValueCounts = schoolData.nunique(dropna=False)
SingleValueCols = UniqueValueCounts[UniqueValueCounts == 1].index
schoolData = schoolData.drop(SingleValueCols, axis=1)

#Review dataset contents after drops
print('*********After: Removing columns with the same value in every row.*******************')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(SingleValueCols))

*********After: Removing columns with the same value in every row.*******************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1472 entries, 2 to 1635
Columns: 571 entries, agency_code to welcome_url
dtypes: float64(528), object(43)
memory usage: 6.4+ MB

Columns Deleted:  914


In [24]:
#Remove any fields that have unique values in every row
schoolDataRecordCt = schoolData.shape[0]
UniqueValueCounts = schoolData.apply(pd.Series.nunique)
AllUniqueValueCols = UniqueValueCounts[UniqueValueCounts == schoolDataRecordCt].index
schoolData = schoolData.drop(AllUniqueValueCols, axis=1)

#Review dataset contents after drops
print('*********After: Removing columns with unique values in every row.*******************')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(AllUniqueValueCols))

*********After: Removing columns with unique values in every row.*******************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1472 entries, 2 to 1635
Columns: 571 entries, agency_code to welcome_url
dtypes: float64(528), object(43)
memory usage: 6.4+ MB

Columns Deleted:  0


In [25]:
#Remove any empty fields (null values in every row)
schoolDataRecordCt = schoolData.shape[0]
NullValueCounts = schoolData.isnull().sum()
NullValueCols = NullValueCounts[NullValueCounts == schoolDataRecordCt].index
schoolData = schoolData.drop(NullValueCols, axis=1)

#Review dataset contents after empty field drops
print('*********After: Removing columns with null / blank values in every row.*************')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(NullValueCols))

*********After: Removing columns with null / blank values in every row.*************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1472 entries, 2 to 1635
Columns: 571 entries, agency_code to welcome_url
dtypes: float64(528), object(43)
memory usage: 6.4+ MB

Columns Deleted:  0


## Handle Other Missing Values Types
* Here we eliminate any numeric columns with more than the percentage of missing values specified by the *missingThreshold* parameter.
* All remaining non-race, numeric column missing values are populated with 0.
* In many cases, it seems that schools are not simply not reporting values when they are zero. However, mean imputation or some other strategy might be considered.

In [26]:
#Isolate continuous and categorical data types
#These are indexers into the schoolData dataframe and may be used similar to the schoolData dataframe 
sD_boolean = schoolData.loc[:, (schoolData.dtypes == bool) ]
sD_nominal = schoolData.loc[:, (schoolData.dtypes == object)]
sD_continuous = schoolData.loc[:, (schoolData.dtypes != bool) & (schoolData.dtypes != object)]
print ("Boolean Columns: ", sD_boolean.shape[1])
print ("Nominal Columns: ", sD_nominal.shape[1])
print ("Continuous Columns: ", sD_continuous.shape[1])
print ("Columns Accounted for: ", sD_nominal.shape[1] + sD_continuous.shape[1] + sD_boolean.shape[1])

Boolean Columns:  0
Nominal Columns:  43
Continuous Columns:  528
Columns Accounted for:  571


In [27]:
#Eliminate continuous columns with more than missingThreshold percentage of missing values
schoolDataRecordCt = sD_continuous.shape[0]
missingValueLimit = schoolDataRecordCt * missingThreshold
NullValueCounts = sD_continuous.isnull().sum()
NullValueCols = NullValueCounts[NullValueCounts >= missingValueLimit].index
schoolData = schoolData.drop(NullValueCols, axis=1)

#Review dataset contents after empty field drops
print('*********After: Removing columns with >= missingThreshold % of missing values******')
schoolData.info(verbose=False)
print('\r\nColumns Deleted: ', len(NullValueCols))

*********After: Removing columns with >= missingThreshold % of missing values******
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1472 entries, 2 to 1635
Columns: 251 entries, agency_code to welcome_url
dtypes: float64(208), object(43)
memory usage: 2.8+ MB

Columns Deleted:  320


## One-Hot Encoding of Categorical Variables
**All categorical / string variables are converted to numberic variables via one hot encoding.  Each unique row value will become a new binary / numeric column in the dataset.**
* All remaining categorical columns are one-hot encoded.  
* In categorical columns, one-hot encoding creates one new boolean / binary field per unique value in the target column, converting all categorical columns to a numeric data type. 
* Prior to one-hot encoding, columns with > *uniqueThreshold* unique values are deleted.  

In [28]:
#Delete categorical columns with > uniqueThreshold unique values 
#(Each unique value becomes a column during one-hot encoding)
oneHotUniqueValueCounts = schoolData[sD_nominal.columns].apply(lambda x: x.nunique())
oneHotUniqueValueCols = oneHotUniqueValueCounts[oneHotUniqueValueCounts >= uniqueThreshold].index
schoolData.drop(oneHotUniqueValueCols, axis=1, inplace=True) 

#Remove categorical columns which do not add value to machine learning
#Grades columns dropped since they all have indicator fields (device_access, device_home, BYOD)
#with less unique values to one-hot encode
schoolData.drop(['welcome_url','device_access_grades',
                 'device_home_grades','byod_grades'], axis=1, inplace=True) 

#Review dataset contents one hot high unique value drops
print('*********After: Removing columns with >= uniqueThreshold unique values***********')
schoolData.info(verbose=False)
print('\r\nColumns Deleted: ', len(oneHotUniqueValueCols))
print(oneHotUniqueValueCols)
print('Dropped Manually: welcome_url,device_access_grades,device_home_grades,byod_grades')

*********After: Removing columns with >= uniqueThreshold unique values***********
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1472 entries, 2 to 1635
Columns: 238 entries, category_code to pct_ready_kea
dtypes: float64(208), object(30)
memory usage: 2.7+ MB

Columns Deleted:  9
Index(['agency_code', 'name', 'street_addr', 'city', 'zip', 'phone', 'url',
       'School Name', 'devices'],
      dtype='object')
Dropped Manually: welcome_url,device_access_grades,device_home_grades,byod_grades


In [29]:
#Isolate remaining categorical variables
begColumnCt = len(schoolData.columns)
sD_nominal = schoolData.loc[:, (schoolData.dtypes == object)]

#one hot encode categorical variables
schoolData = pd.get_dummies(data=schoolData, 
                       columns=sD_nominal.columns, drop_first=True)

#Determine change in column count
endColumnCt = len(schoolData.columns)
columnsAdded = endColumnCt - begColumnCt

#Review dataset contents one hot high unique value drops
print('Columns To One-Hot Encode: ', len(sD_nominal.columns))
print('\r\n*********After: Adding New Columns Via One-Hot Encoding*************************')
schoolData.info(verbose=False)
print('\r\nNew Columns Created Via One-Hot Encoding: ', columnsAdded)

Columns To One-Hot Encode:  30

*********After: Adding New Columns Via One-Hot Encoding*************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1472 entries, 2 to 1635
Columns: 632 entries, IndianPct to BYOD_Yes
dtypes: float64(208), uint8(424)
memory usage: 2.9 MB

New Columns Created Via One-Hot Encoding:  394


In [30]:
#Remove any fields that have the same value in all rows after one-hot encoding
UniqueValueCounts = schoolData.nunique(dropna=False)
SingleValueCols = UniqueValueCounts[UniqueValueCounts == 1].index
schoolData = schoolData.drop(SingleValueCols, axis=1)

#Review dataset contents after drops
print('*********After: Removing one-hot encoded columns with the same value in every row.*******************')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(SingleValueCols))

*********After: Removing one-hot encoded columns with the same value in every row.*******************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1472 entries, 2 to 1635
Columns: 632 entries, IndianPct to BYOD_Yes
dtypes: float64(208), uint8(424)
memory usage: 2.9 MB

Columns Deleted:  0


## Impute any Remaining Missing Values as Zero

In [31]:
#Print out all the missing value rows
pd.set_option('display.max_rows', 1000)

print('\r\n*********The Remaining Missing Values Below will be set to Zero!*************************')

#Check for Missing values 
missing_values = schoolData.isnull().sum().reset_index()
missing_values.columns = ['Variable Name', 'Number Missing Values']
missing_values = missing_values[missing_values['Number Missing Values'] > 0] 
missing_values




*********The Remaining Missing Values Below will be set to Zero!*************************


,Variable Name,Number Missing Values
0,IndianPct,63
1,AsianPct,63
2,HispanicPct,63
3,BlackPct,63
4,WhitePct,63
5,PacificIslandPct,63
6,TwoOrMorePct,63
7,MinorityPct,63
8,IndianMalePct,63
9,AsianMalePct,63


In [32]:
#Replace all remaining NaN with 0
schoolData = schoolData.fillna(0)

#Check for Missing values after final imputation 
missing_values = schoolData.isnull().sum().reset_index()
missing_values.columns = ['Variable Name', 'Number Missing Values']
missing_values = missing_values[missing_values['Number Missing Values'] > 0] 
missing_values

,Variable Name,Number Missing Values


## Identify and Remove Highly Correlated Features
**Find and remove any columns / features that are > 95% correlated**
* https://stackoverflow.com/questions/39409866/correlation-heatmap
* https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/
* https://codeyarns.com/2015/04/20/how-to-change-font-size-in-seaborn/

In [33]:
# calculate the correlation matrix
corr_matrix  = schoolData.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [34]:
#Get all of the correlation values > 95%
x = np.where(upper > 0.95)

#Display all field combinations with > 95% correlation
cf = pd.DataFrame()
cf['Field1'] = upper.columns[x[1]]
cf['Field2'] = upper.index[x[0]]

#Get the correlation values for every field combination. (There must be a more pythonic way to do this!)
corr = [0] * len(cf)
for i in range(0, len(cf)):
    corr[i] =  upper[cf['Field1'][i]][cf['Field2'][i]] 
    
cf['Correlation'] = corr

print ('There are ', str(len(cf['Field1'])), ' field correlations > 95%.')
cf

There are  202  field correlations > 95%.


,Field1,Field2,Correlation
0,IndianMalePct,IndianPct,0.995949
1,IndianFemalePct,IndianPct,0.995642
2,AsianMalePct,AsianPct,0.966859
3,AsianFemalePct,AsianPct,0.969315
4,HispanicMalePct,HispanicPct,0.969134
5,HispanicFemalePct,HispanicPct,0.968803
6,BlackMalePct,BlackPct,0.986823
7,BlackFemalePct,BlackPct,0.984954
8,WhiteMalePct,WhitePct,0.986655
9,WhiteFemalePct,WhitePct,0.984885


In [35]:
#The old way
#print('Dropping the following ', str(len(to_drop)), ' highly correlated fields.')
#to_drop

# Comparing highly correlated unique field names in field 1 vs field 2 list above will allow you to 
# remove the least amount of fields possible.  
#len(cf['Field1'].unique())
#len(cf['Field2'].unique())

print('Dropping the following ', str(len(cf['Field2'].unique())), ' highly correlated, unique fields.')
cf['Field2'].unique()

Dropping the following  151  highly correlated, unique fields.


array(['IndianPct', 'AsianPct', 'HispanicPct', 'BlackPct', 'WhitePct',
       'MinorityPct', 'IndianMalePct', 'eg_index_ALL_ALL',
       'eg_index_MA_ALL', 'eg_score_ALL_ALL', 'eg_score_ALL_BL7',
       'eg_score_ALL_EDS', 'eg_score_ALL_ELS', 'eg_score_ALL_HI7',
       'eg_score_ALL_SWD', 'eg_score_ALL_WH7', 'eg_score_RD_ALL',
       'pct_ALL_ELP', 'pct_met_Math Grades 3-8_PART',
       'pct_met_Reading Grades 3-8_PART',
       'pct_met_Science Grades 5&8_PART', 'pct_met_Total Targets_PART',
       'pct_Math Grades 3-8_ALL_PART_DET',
       'pct_Math Grades 3-8_BL7_PART_DET',
       'pct_Math Grades 3-8_EDS_PART_DET',
       'pct_Math Grades 3-8_HI7_PART_DET',
       'pct_Math Grades 3-8_SWD_PART_DET',
       'pct_Math Grades 3-8_WH7_PART_DET',
       'pct_Science Grades 5&8_WH7_PART_DET', 'ma_score', 'rd_score',
       'ach_score', 'eg_score', 'spg_score', 'ma_spg_score',
       'aaa_score_ALL_SPG2', 'aaa_score_BL7_SPG2', 'aaa_score_EDS_SPG2',
       'aaa_score_ELS_SPG2', 'aaa_score_H

In [36]:
#Check columns before drop 
print('\r\n*********Before: Dropping Highly Correlated Fields*************************************')
schoolData.info(verbose=False)

# Drop the highly correlated features from our training data 
#schoolData = schoolData.drop(to_drop, axis=1)
schoolData = schoolData.drop(cf['Field2'], axis=1)

#Check columns after drop 
print('\r\n*********After: Dropping Highly Correlated Fields**************************************')
schoolData.info(verbose=False)


*********Before: Dropping Highly Correlated Fields*************************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1472 entries, 2 to 1635
Columns: 632 entries, IndianPct to BYOD_Yes
dtypes: float64(208), uint8(424)
memory usage: 2.9 MB

*********After: Dropping Highly Correlated Fields**************************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1472 entries, 2 to 1635
Columns: 481 entries, PacificIslandPct to BYOD_Yes
dtypes: float64(149), uint8(332)
memory usage: 2.2 MB


In [37]:
#Restore the unit_code before saving
schoolData['agency_code'] = agency_code
#Save the final dataset to a .csv file
schoolData.to_csv(outputDir + inputFileName + '_ML.csv', sep=',', index=False)

In [38]:
print('*********FINAL DATASET DETAILS*********************************************************\r\n')
schoolData.info(verbose=True)

*********FINAL DATASET DETAILS*********************************************************

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1472 entries, 2 to 1635
Data columns (total 482 columns):
PacificIslandPct                                                                                      float64
TwoOrMorePct                                                                                          float64
AsianMalePct                                                                                          float64
HispanicMalePct                                                                                       float64
BlackMalePct                                                                                          float64
WhiteMalePct                                                                                          float64
PacificIslandMalePct                                                                                  float64
TwoOrMoreMalePct                 

In [39]:
import sklearn
import pandas as pd

print('Sklearn Version: ' + sklearn.__version__)
print('Pandas Version: ' + pd.__version__)

Sklearn Version: 0.21.2
Pandas Version: 0.23.4


In [40]:
print('Output File Location:\r\n\r\n' + outputDir + inputFileName + '_ML.csv')

Output File Location:

C:/Users/Jake/Documents/GitHub/EducationDataNC/2019/Machine Learning Datasets/PublicElementarySchools2019_ML.csv
